# Code Optimization [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ua-2025q3-astr501-513/ua-2025q3-astr501-513.github.io/blob/main/501/05/lab.ipynb)

Speeding Up Your Python Codes 1000x!!!

## Introduction

Welcome to "Code Optimization: Speeding Up Your Python Codes 1000x"!

In the next hours, we will learn tricks to unlock python performance
and increase a sample code's speed by a factor of 1000x!
This lab is based on a Hack Arizona tutorial CK gave in 2024.
The [original lab](https://github.com/rndsrc/orbits-py) can be found
here.

Here is our plan:
* Understand the problem:
  we will start by outlining the $n$-body problem and how to use the
  leapfrog algorithm to solve it numerically.
* Benchmark and profile:
  we will learn how to measure performance and identify bottlenecks
  using tools like timeit and cProfile.
* Optimize the code:
  step through a series of optimizations:
  * Use list comprehensions and reduce operation counts.
  * Replace slow operations with faster ones.
  * Use high-performance libraries such as NumPy.
  * Explore lower precision where applicable.
  * Use Google JAX for just-in-time compilation and GPU acceleration.
* Apply the integrator:
  finally, we will use our optimized code to simulate the mesmerizing
  figure-8 self-gravitating orbit.

By the end of this lab, you will see that Python isn't just great for
rapid prototyping.
It can also deliver serious performance when optimized right.
Let's dive in!

## The $n$-body Problem and the Leapfrog Algorithm

The $n$-body problem involves simulating the motion of many bodies
interacting with each other through (usually) gravitational forces.

This problem is physically interesting, as it models the
[solar system](https://rebound.readthedocs.io/en/latest/),
[galaxy dynamics, or even the whole cosmic structure
formation](https://wwwmpa.mpa-garching.mpg.de/gadget/)!

This problem is also computational interesting.
Each body experiences a force from every other body, making the
simulation computationally intensive.
It is ideal for exploring optimization techniques.

The leapfrog algorithm is a simple, robust numerical integrator that
alternates between updating velocities ("kicks") and positions
("drifts").
It is popular because it conserves energy and momentum well over long
simulation periods.

We need to solve Newton's second law of motion for a collection of $n$
bodies.
The body forces are solved by using Newton's Law of Universal
Gravitation.

### Gravitational Force/Acceleration Equation

For any two bodies labeled by $i$ and $j$, the direct gravitational
force exerted on body $i$ by body $j$ is given by:
\begin{align}
  \mathbf{f}_{ij} = -G m_i m_j\frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^{3/2}},
\end{align}
where:
* $G$ is the gravitational constant.
* $m_i$ and $m_j$ are the masses of the bodies.
* $\mathbf{r}_i$ and $\mathbf{r}_j$ are their position vectors.

Summing the contributions from all other bodies gives the net force
(and thus the acceleration) on body $i$:
\begin{align}
  \mathbf{f}_i
  = \sum_{j\ne i} \mathbf{f}_{ij}
  = -G m_i \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3},
\end{align}

Given Newton's law $\mathbf{f} = m\mathbf{a}$, the "acceleration"
applied on body $i$ caused by all other bodies is:
\begin{align}
  \mathbf{a}_i
  = \sum_{j\ne i} \mathbf{a}_{ij}
  = -G \sum_{j\ne i} m_j \frac{\mathbf{r}_i - \mathbf{r}_j\ \ \ }{|\mathbf{r}_i - \mathbf{r}_j|^3}.
\end{align}
Choosing the right units so $G = 1$.

Here is a pure Python implementation of the gravitational
acceleration:

In [ ]:
def acc1(m, r):
    
    n = len(m)
    a = []
    for i in range(n):
        axi, ayi, azi = 0, 0, 0
        for j in range(n):
            if j != i:
                xi, yi, zi = r[i]
                xj, yj, zj = r[j]

                axij = - m[j] * (xi - xj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                ayij = - m[j] * (yi - yj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)
                azij = - m[j] * (zi - zj) / ((xi - xj)**2 + (yi - yj)**2 + (zi - zj)**2)**(3/2)

                axi += axij
                ayi += ayij
                azi += azij

        a.append((axi, ayi, azi))

    return a

We may try using this function to evaluate the gravitational force
between two particles, with with mass $m = 1$, at a distance of $2$.
We expect the "acceleration" be $-1/4$ in the direction of their
seperation.

In [ ]:
m  = [1.0, 1.0]
r0 = [
    (+1.0, 0.0, 0.0),
    (-1.0, 0.0, 0.0),
]
a0ref = [
    (-0.25, 0.0, 0.0),
    (+0.25, 0.0, 0.0),
]

In [ ]:
a0 = acc1(m, r0)

In [ ]:
a0

In [ ]:
assert a0 == a0ref